In [1]:
!nvidia-smi

Wed Aug 12 20:41:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

Cloning into 'dnn_from_scratch'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 1432 (delta 137), reused 148 (delta 83), pack-reused 1218
Receiving objects: 100% (1432/1432), 13.74 MiB | 3.97 MiB/s, done.
Resolving deltas: 100% (952/952), done.


In [3]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning_from_scratch'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 126 (delta 71), reused 97 (delta 42), pack-reused 0
Receiving objects: 100% (126/126), 77.38 KiB | 200.00 KiB/s, done.
Resolving deltas: 100% (71/71), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning_from_scratch


In [2]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/ShivamShrirao/deep_Q_learning_from_scratch
   8fdfa27..6f82c89  master     -> origin/master
Updating 8fdfa27..6f82c89
Fast-forward
 agent.py | 9 +++++----
 1 file changed, 5 insertions(+), 4 deletions(-)


In [8]:
import gym
import time
import cv2
import numpy as np

from settings import *
from agent import *
from experience import *
from atari_wrappers import *

In [4]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-6)
D_exp = ReplayMemory(capacity=600000)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 BatchNormalization(Batc (None, 40, 40, 32)         echo             128
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
3 BatchNormalization(Batc (None, 20, 20, 64)         echo             256
__________________________________________________________________________________________
4 Con

In [5]:
env = gym.make('Pong-v0')
env = FrameStack(env, NFRAMES)      # preprocess and stack frames

In [6]:
len(D_exp.ndone)

0

In [ ]:
for i_episode in range(1000):
    observation = env.reset()
    ep_score = 0
    start = time.time()
    nlap = NFRAMES - 2
    sv_obs = deque([], maxlen=nlap)
    sv_act = deque([], maxlen=nlap)
    sv_rew = deque([], maxlen=nlap)
    sv_don = deque([], maxlen=nlap)
    for t in range(10000):
#         env.render()
        action = agt.get_action(observation)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        sv_obs.append(observation)
        sv_act.append(action)
        sv_rew.append(reward)
        sv_don.append(done)

        if t >= nlap-1:
            D_exp.store_transition(sv_obs[0], agt.actions.index(sv_act[0]), sv_rew[0], next_observation, sv_don[0])
        observation = next_observation

        # D_exp.current_state.popleft()
        # D_exp.action_idx.popleft()
        # D_exp.reward.popleft()
        # D_exp.next_state.popleft()
        # D_exp.ndone.popleft()

        if len(D_exp.ndone) > BATCH_SIZE:
            agt.trainDDQN(D_exp, BATCH_SIZE)
            agt.update_target()
        if done:
            D_exp.store_transition(sv_obs[-1], agt.actions.index(sv_act[-1]), sv_rew[-1], next_observation, sv_don[-1])
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon}, Time: {time.time()-start}")
    if not i_episode%4:
        agt.model.save_weights("/content/drive/My Drive/model.w8s")
env.close()

In [ ]:
env.close()